In [1]:
import pandas as pd
#from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import torch
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('data.xlsx') 

In [4]:
df = df[['摘要(全英)', '标题(全英)',]]  
df

,摘要(全英),标题(全英),发明人(全英)
0,The problem solved by the present invention is...,Fertilizer absorption improvement method,SATO Takashi
1,The present invention relates to a pesticide n...,pesticide,MURPHY BROOK C. | SOON TEO ENG
2,Microparticle constructs stabilized by an amph...,Technical forms of structural liberalization,"MAYER, LAWRENCE, D. | PRUD'HOMME, ROBERT, K. |..."
3,The present invention provides antimicrobial c...,New synthetic antimicrobial peptides,"KENDURKAR, SHUCHISHWETA VINAY | SENGUPTA, DURBA"
4,carbon nanotube production system is used to i...,Microbial stimulation and engine emission limits,"LEWIS, GARY"
...,...,...,...
2588,"Many active agents, such as pesticides, herbic...",Pesticide Nanosuspension,"MISSELBROOK, JOHN | DUNN, JEFF"
2589,The present invention relates to a pesticide n...,pesticide,"TEO, ENG SOON | MURPHY, BROOK C"
2590,The present invention discloses a nano PCR det...,Infectious bovine rhinotracheitis virus nano-P...,"GUO, LI | WANG, ZHENJUN | ZHAO, LIFENG | LUO, ..."
2591,The present invention provides a graphene-like...,Graphene-like biochar loaded with nano zero-va...,FANG LIPING | LIU KAI | RAN QIWANG | MA YIBING


In [6]:
# Define a function to remove special characters
def truncate_text(text, max_length=500, redundancy=20):
    if len(text) <= max_length:
        return text
    
    # find the last period within the max length
    end_idx_cn = text.rfind('。', 0, max_length)
    # find the last period within the max length
    end_idx_en = text.rfind('.', 0, max_length)
    
    end_idx = max(end_idx_cn, end_idx_en)
    
    if end_idx == -1:
        # If no period is found, force the cut and retain the redundancy
        start_idx = max_length - redundancy if max_length > redundancy else 0
        end_idx = max_length + redundancy if len(text) > max_length + redundancy else len(text)
        return text[start_idx:end_idx]
    else:
        return text[:end_idx + 1]

df['摘要(全英)'] = df['摘要(全英)'].apply(lambda x: truncate_text(x, max_length=500))

In [7]:
abstract = df['摘要(全英)'].tolist()
title = df['标题(全英)'].tolist()
abstracts = title + abstract

In [8]:
data = abstract
data

['The problem solved by the present invention is to provide a method for improving fertilizer absorption which can conveniently improve fertilizer absorption. The method for improving fertilizer absorption involves applying nanobubble water to plants.',
 'The present invention relates to a pesticide nanoemulsion, comprising: (a) horticultural oil as an active ingredient; (b) a mixture of inert ingredients, including: (i) an emulsifier; (ii) a stabilizer; (iii) a surfactant; and (c) a solvent, wherein the horticultural oil is dispersed in the solvent in the form of droplets to form a nanoemulsion.',
 'Microparticle constructs stabilized by an amphiphilic copolymer and comprising at least one active substance coupled to a hydrophobic moiety provide sustained release of the active substance in both in vitro and in vivo environments.',
 'The present invention provides antimicrobial compositions comprising synthetic peptides for therapeutic use. These compositions have been found to be usef

In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained model and tokenizer
tokenizer = DistilBertTokenizer.from_pretrained(
    r'C:\Users\RIC_ZX\Desktop\papers\ZLW\SBERT\NEW\distilbert-base-nli-mean-tokens'
)
model = DistilBertModel.from_pretrained(
    r'C:\Users\RIC_ZX\Desktop\papers\ZLW\SBERT\NEW\distilbert-base-nli-mean-tokens'
).to(device)

# Hyperparameters
lr = 0.00002
batch_size = 16
num_epochs = 30

# Optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
loss_fn = torch.nn.BCEWithLogitsLoss()

# Fine-tuning loop
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for i in range(0, len(data), batch_size):
        # Sample positive example
        pos_text = data[i]

        # Sample negative example
        neg_text_index = torch.randint(len(data), (1,)).item()
        while neg_text_index == i:
            neg_text_index = torch.randint(len(data), (1,)).item()
        neg_text = data[neg_text_index]

        # Tokenize and move data to GPU
        pos_inputs = tokenizer(pos_text, return_tensors="pt", padding=True, truncation=True).to(device)
        neg_inputs = tokenizer(neg_text, return_tensors="pt", padding=True, truncation=True).to(device)

        # Generate embeddings
        pos_vector = model(**pos_inputs).last_hidden_state.mean(dim=1)
        neg_vector = model(**neg_inputs).last_hidden_state.mean(dim=1)

        # Calculate similarity
        similarity = torch.cosine_similarity(pos_vector, neg_vector).mean()

        # Create labels (1 for similar, 0 for dissimilar)
        labels = torch.tensor([1.0 if similarity.item() >= 0 else 0.0], dtype=torch.float).to(device)

        # Compute loss
        outputs = torch.sum(pos_vector * neg_vector).unsqueeze(0)  # Dot product as logits
        loss = loss_fn(outputs, labels)
        epoch_loss += loss.item()

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} loss: {epoch_loss:.4f}")

In [11]:
torch.save(model.state_dict(), 'distilbert.pth')